In [1]:
import pandas as pd
import numpy as np

# Read the CSV files into DataFrames
df_user = pd.read_csv('../2024初賽用開放資料/user_data.csv')
df_exam = pd.read_csv('../2024初賽用開放資料/dp001_exam.csv')
df_prac = pd.read_csv('../2024初賽用開放資料/dp001_prac.csv')
df_review = pd.read_csv('../2024初賽用開放資料/dp001_review.csv')
df_review_plus = pd.read_csv('../2024初賽用開放資料/dp001_review_plus.csv')
df_exam2 = pd.read_csv('../2024初賽用開放資料/dp002_exam.csv')
df_math = pd.read_csv('../2024初賽用開放資料/dp003_math.csv')
df_word = pd.read_csv('../2024初賽用開放資料/dp003_word.csv')



In [2]:
df_user = pd.read_csv('../2024初賽用開放資料/user_data.csv')
df_exam = pd.read_csv('../2024初賽用開放資料/dp001_exam.csv')
df_prac = pd.read_csv('../2024初賽用開放資料/dp001_prac.csv')
df_review = pd.read_csv('../2024初賽用開放資料/dp001_review.csv')
df_review_plus = pd.read_csv('../2024初賽用開放資料/dp001_review_plus.csv')
df_exam2 = pd.read_csv('../2024初賽用開放資料/dp002_exam.csv')
df_math = pd.read_csv('../2024初賽用開放資料/dp003_math.csv')
df_word = pd.read_csv('../2024初賽用開放資料/dp003_word.csv')



### COMBINE Data Frame

In [3]:
df_user_review = df_user.merge(df_review, on='user_sn', how='left')
df_combined = df_user_review.merge(df_review_plus, on='review_sn', how='left')
df_filtered = df_combined[~df_combined['review_sn'].isin(df_exam['review_sn'])]

df_filtered.to_csv('df_dp001_review.csv', index=False)


### 分群準備
#### df_filter 
合併後有缺值是因為並非每一位學生都有做dp001的練習題, 以及觀看dp001平台的影片

In [4]:
df_filtered.info()

missing_data = df_filtered.isnull().sum()
print("缺失值\n", missing_data[missing_data > 0])


<class 'pandas.core.frame.DataFrame'>
Index: 19495 entries, 0 to 70100
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   user_sn          19495 non-null  int64  
 1   organization_id  19495 non-null  int64  
 2   grade            19495 non-null  int64  
 3   class            19495 non-null  int64  
 4   seat             19495 non-null  int64  
 5   chinese_score    19495 non-null  int64  
 6   math_score       19495 non-null  int64  
 7   english_score    19495 non-null  int64  
 8   review_sn        19458 non-null  float64
 9   indicator_name   19458 non-null  object 
 10  subject_name     19458 non-null  object 
 11  video_name       19458 non-null  object 
 12  video_len        19458 non-null  float64
 13  start_timestamp  19458 non-null  float64
 14  start_time       19458 non-null  object 
 15  end_timestamp    19458 non-null  float64
 16  end_time         19458 non-null  object 
 17  finish_rate      

In [5]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

# 假設 behavior_counts_scaled 是一個 NumPy 陣列或 pandas DataFrame
# 檢查是否有 NaN 值
#if np.isnan(behavior_counts_scaled).any():
    #raise ValueError("behavior_counts_scaled 包含 NaN 值，請處理後再進行聚類。")


In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# 1. 準備特徵工程
def extract_learning_behaviors(df):
    # 計算每位學生的學習行為特徵
    behavior_features = df.groupby('user_sn').agg({
        # 重播次數
        'review_sn': 'count', 
        
        # 完成率統計
        'finish_rate': ['mean', 'std'],
        
        # 視頻操作行為
        'view_action': lambda x: x.value_counts().get('pause', 0),
        
        # 時間相關特徵
        'start_timestamp': lambda x: np.max(x) - np.min(x),
        'end_timestamp': lambda x: np.max(x) - np.min(x),
        
        # 影片長度相關
        'video_len': 'mean',
        
        # 學習成績
        'chinese_score': 'first',
        'math_score': 'first',
        'english_score': 'first'
    }).reset_index()
    
    # 重命名列
    behavior_features.columns = [
        'user_sn', 
        'review_count', 
        'avg_finish_rate', 
        'finish_rate_std', 
        'pause_count', 
        'learning_duration', 
        'video_end_duration',
        'avg_video_length', 
        'chinese_score', 
        'math_score', 
        'english_score'
    ]
    
    return behavior_features


In [7]:

# 2. 特徵標準化
def preprocess_features(features):
    # 排除user_sn列
    X = features.drop('user_sn', axis=1)
    
    # 填補缺失值
    X = X.fillna(X.mean())
    
    # 標準化
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    return X_scaled, X.columns


In [8]:

# 3. 選擇最佳聚類數
def find_optimal_clusters(X_scaled, max_clusters=10):
    silhouette_scores = []
    
    for n_clusters in range(2, max_clusters + 1):
        kmeans = KMeans(
            n_clusters=n_clusters, 
            n_init=10, 
            random_state=42
        )
        kmeans.fit(X_scaled)
        score = silhouette_score(X_scaled, kmeans.labels_)
        silhouette_scores.append(score)
    
    optimal_clusters = silhouette_scores.index(max(silhouette_scores)) + 2
    return optimal_clusters


In [9]:

# 4. 執行K-means分群
def perform_clustering(df):
    # 提取特徵
    behavior_features = extract_learning_behaviors(df)
    
    # 特徵標準化
    X_scaled, feature_names = preprocess_features(behavior_features)
    
    # 找最佳聚類數
    optimal_clusters = find_optimal_clusters(X_scaled)
    
    # 執行K-means
    kmeans = KMeans(
        n_clusters=optimal_clusters, 
        n_init=10, 
        random_state=42
    )
    cluster_labels = kmeans.fit_predict(X_scaled)
    
    # 將分群結果加入結果
    result = behavior_features.copy()
    result['cluster'] = cluster_labels
    
    return result, kmeans, X_scaled, feature_names


In [10]:

# 主執行
def main():
    # 使用您之前準備的 df_filtered
    clustering_result, kmeans_model, scaled_features, feature_names = perform_clustering(df_filtered)
    
    # 分析每個群體的特徵
    cluster_summary = clustering_result.groupby('cluster').agg({
        'review_count': 'mean',
        'avg_finish_rate': 'mean', 
        'pause_count': 'mean',
        'chinese_score': 'mean',
        'math_score': 'mean',
        'english_score': 'mean'
    })
    
    print("最佳聚類數:", len(cluster_summary))
    print("\n各群體特徵:\n", cluster_summary)
    
    # 儲存結果
    clustering_result.to_csv('student_learning_clusters.csv', index=False)
    
    return clustering_result, cluster_summary

# 執行主程式
clustering_result, cluster_summary = main()

最佳聚類數: 6

各群體特徵:
          review_count  avg_finish_rate  pause_count  chinese_score  \
cluster                                                              
0          593.941176        70.669700          0.0      72.941176   
1           60.428571        59.386071          0.0      75.619048   
2           17.333333        61.952381          0.0      81.333333   
3          186.636364        62.048206          0.0      80.000000   
4          438.000000        84.183869          0.0      56.750000   
5          164.777778        57.109273          0.0      42.111111   

         math_score  english_score  
cluster                             
0         50.823529      86.352941  
1         50.095238      80.523810  
2         61.333333      98.000000  
3         57.818182      92.363636  
4         39.000000      40.750000  
5         27.777778      41.000000  
